In [2]:
from itertools import takewhile, starmap
from functools import partial, reduce
 
def compose_two(g, f):
    """Function composition for two functions, e.g. compose_two(f, g)(x) == f(g(x))"""
    return lambda *args, **kwargs: g(f(*args, **kwargs))
 
def compose(*funcs):
    """Compose an arbitrary number of functions left-to-right passed as args"""
    return reduce(compose_two, funcs)
 
def transform_args(func, transformer):
    return lambda *args: func(*transformer(args))
 
composed_partials = transform_args(compose, partial(starmap, partial))
pipe = transform_args(composed_partials, reversed)
 
pipe_style = pipe(
    (takewhile, lambda x: x < 7),
    (filter, lambda x: x < 2),
    (map, lambda x: 4 * x))

In [2]:
list(pipe_style(range(100)))

[0, 4]

In [6]:
from sklearn.preprocessing import Normalizer, LabelBinarizer,RobustScaler,StandardScaler, OneHotEncoder

In [3]:
import numpy as np
import pandas as pd

In [4]:
df_train = pd.read_json(open("./data/train.json", "r"))
df_test = pd.read_json(open("./data/test.json", "r"))

In [6]:
from multiprocessing import Pool, cpu_count

In [5]:
selection = [
    (df_train, df_test,["bathrooms"],RobustScaler()),
    (df_train, df_test, ["bedrooms"],RobustScaler()),
    #(df_train,"latitude",None),
    #(df_train,"longitude",None),
    (df_train, df_test, ["price"],RobustScaler())
    # (df_train,["interest_level"],LabelBinarizer())
    ]

NameError: name 'RobustScaler' is not defined

In [8]:
def transformation(train,test,sCol,Transformer=None):
    if Transformer is None:
        return train, test
    
    def _trans(df, sCol, Transformer,flag):
        if flag == "fit_transform":
            transformed = Transformer.fit_transform(df[sCol]).T
        elif flag == "transform":
            transformed = Transformer.transform(df[sCol]).T
        elif flag == "fit":
            transformed = Transformer.fit(df[sCol]).T
        #else Raise exception
        
        if isinstance(sCol, list):
            label = sCol[0]
        else:
            label = sCol
        
        # To keep the index, we need to assign to the original dataframe and then extract again
        # OneHotEncoder and LabelBinarizer do not have the same interface ....
        feature_list = []
        n = 0
        feat_label = ''
        for serie in transformed:
            if isinstance(Transformer, OneHotEncoder):
                feat_label = label + '_' + Transformer.active_features_[n]
            else:
                try:
                    feat_label = label + '_' + Transformer.classes_[n]
                except:
                    feat_label = label
            df[feat_label] = serie
            feature_list.append(feat_label)
        return df[feature_list]
    
    trn = train.copy()
    tst = test.copy()
    
    trn = _trans(trn, sCol, Transformer,'fit_transform')
    tst = _trans(tst, sCol, Transformer,'transform')
    
    #We only send back what we need
    return (trn, tst)

In [10]:
result = pd.DataFrame()
result

""


In [11]:
d, t = transformation(df_train,["interest_level"],LabelBinarizer())

In [12]:
d, t

(        bathrooms  bedrooms                       building_id  \
 10            1.5         3  53a5b119ba8f7b61d4e010512e0dfc85   
 10000         1.0         2  c5c8a357cba207596b04d1afd1e4f130   
 100004        1.0         1  c3ba40552e2120b0acfc3cb5730bb2aa   
 100007        1.0         1  28d9ad350afeaab8027513a3e52ac8d5   
 100013        1.0         4                                 0   
 100014        2.0         4  38a913e46c94a7f46ddf19b756a9640c   
 100016        1.0         2  3ba49a93260ca5df92fde024cb4ca61f   
 100020        2.0         1  0372927bcb6a0949613ef5bf893bbac7   
 100026        1.0         1  a7efbeb58190aa267b4a9121cd0c88c0   
 100027        2.0         4                                 0   
 100030        1.0         0                                 0   
 10004         1.0         1                                 0   
 100044        1.0         2  67c9b420da4a365bc26a6cd0ef4a5320   
 100048        2.0         2                                 0   
 10005    

In [ ]:
with Pool(cpu_count()) as mp:
    def zip_with(f, list_of_tuple):
        return mp.starmap(f, zip(*list_of_tuple))
    def vconcat(*list_of_df):
        return pd.concat([*list_of_df], axis=1)
    
    ltuples_train_test = mp.starmap(transformation,selection)
    train, test = zip_with(vconcat,ltuples_train_test)
    

Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 345, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'vconcat' on <module '__main__'>
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 345, in get
    return _ForkingPi

In [ ]:
[(df_train,df_test,label,transfo) for (label,transfo) in test2]

In [ ]:
    test3 = [
    (["bathrooms"],RobustScaler(),df_train,df_test),
    (["bedrooms"],RobustScaler(),df_train,df_test),
    #("latitude",None,df_train,df_test),
    #("longitude",None,df_train,df_test),
    (["price"],RobustScaler(),df_train,df_test)
    ]

In [ ]:
test3

In [7]:
from functools import reduce
from itertools import starmap
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# zipWith
def zip_with(f, list_of_tuple):
    return starmap(f, zip(*list_of_tuple))

############# Helper pandas ###################
def vconcat(*list_of_df):
    return pd.concat([*list_of_df], axis=1)

In [8]:
def feat_selection(ft_selection,df_train,df_test):
    trn = df_train.copy()
    # val = df_val.copy()
    tst = df_test.copy()
    
    print(ft_selection)
    
    # ft_selection = [(trn,val,tst,label,transfo) for (label,transfo) in ft_selection]
    ft_selection = [(trn,tst,label,transfo) for (label,transfo) in ft_selection]
    
    tuples_trn_val_test = starmap(feat_transformation,ft_selection)
    train, test = zip_with(vconcat,tuples_trn_val_test)
    
    # Multiprocessing is broken
    # with Pool(cpu_count()) as mp:
    #    tuples_trn_val_test = mp.starmap(feat_transformation,ft_selection)
    #    train, valid, test = par_zip_with(vconcat,tuples_trn_val_test, mp)
    return train, test

def feat_transformation(train, test,sCol,Transformer=None):
#def feat_transformation(train,valid, test,sCol,Transformer=None):
    if Transformer is None:
        #return (train, valid, test)
        return (train, test)
    
    def _trans(df, sCol, Transformer,flag):
        if flag == "fit_transform":
            transformed = Transformer.fit_transform(df[sCol]).T
        elif flag == "transform":
            transformed = Transformer.transform(df[sCol]).T
        elif flag == "fit":
            transformed = Transformer.fit(df[sCol]).T
        #else Raise exception
        
        if isinstance(sCol, list):
            label = sCol[0]
        else:
            label = sCol
        
        # To keep the index, we need to assign to the original dataframe and then extract again
        # OneHotEncoder and LabelBinarizer do not have the same interface ....
        feature_list = []
        n = 0
        feat_label = ''
        for serie in transformed:
            if isinstance(Transformer, OneHotEncoder):
                feat_label = label + '_' + Transformer.active_features_[n]
            else:
                try:
                    feat_label = label + '_' + Transformer.classes_[n]
                except:
                    feat_label = label
            df[feat_label] = serie
            feature_list.append(feat_label)
        return df[feature_list]
    
    trn = train.copy()
    #val = valid.copy()
    tst = test.copy()
    
    trn = _trans(trn, sCol, Transformer,'fit_transform')
    #val = _trans(val, sCol, Transformer,'transform')
    tst = _trans(tst, sCol, Transformer,'transform')
    # If you have unseen data issue when transforming test, preprocess your data, don't leak. (Label don't exist in train
    
    #We only send back what we need
    #return (trn, val, tst)
    return (trn, tst)

In [9]:
df_train = pd.read_json(open("./data/train.json", "r"))
df_test = pd.read_json(open("./data/test.json", "r"))

In [11]:
# d, t = transformation(df_train,["interest_level"],LabelBinarizer())
p = feat_transformation(df_train,df_test,["price"],RobustScaler())
ba = feat_transformation(df_train,df_test,["bathrooms"],RobustScaler())

In [15]:
p[1]

,price
0,-0.125000
1,-0.187500
100,0.380000
1000,0.093750
100000,1.093750
100001,3.656250
100002,-0.218750
100003,-0.781250
100005,-0.093750
100006,-0.531250


In [16]:
 sel = [
    (["bathrooms"],RobustScaler()),
    (["bedrooms"],RobustScaler())
 ]

In [24]:
a,b = feat_selection(sel, df_train, df_test)

[(['bathrooms'], RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), (['bedrooms'], RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True))]


In [25]:
a

,bathrooms,bedrooms
10,0.5,2.0
10000,0.0,1.0
100004,0.0,0.0
100007,0.0,0.0
100013,0.0,3.0
100014,1.0,3.0
100016,0.0,1.0
100020,1.0,0.0
100026,0.0,0.0
100027,1.0,3.0


In [28]:
def feat_selection(ft_selection,df_train,df_val,df_test):
    trn = df_train.copy()
    val = df_val.copy()
    tst = df_test.copy()
    
    print(ft_selection)
    
    ft_selection = [(trn,val,tst,label,transfo) for (label,transfo) in ft_selection]
    
    tuples_trn_val_test = starmap(feat_transformation,ft_selection)
    trn, val, tst = zip_with(vconcat,tuples_trn_val_test)
    
    # Multiprocessing is broken
    # with Pool(cpu_count()) as mp:
    #    tuples_trn_val_test = mp.starmap(feat_transformation,ft_selection)
    #    train, valid, test = par_zip_with(vconcat,tuples_trn_val_test, mp)
    return trn, val, tst

def feat_transformation(train,valid, test,sCol,Transformer=None):
    if Transformer is None:
        return (train, valid, test)

    def _trans(df, sCol, Transformer,flag):
        if flag == "fit_transform":
            transformed = Transformer.fit_transform(df[sCol]).T
        elif flag == "transform":
            transformed = Transformer.transform(df[sCol]).T
        elif flag == "fit":
            transformed = Transformer.fit(df[sCol]).T
        #else Raise exception
        
        if isinstance(sCol, list):
            label = sCol[0]
        else:
            label = sCol
        
        # To keep the index, we need to assign to the original dataframe and then extract again
        # OneHotEncoder and LabelBinarizer do not have the same interface ....
        feature_list = []
        n = 0
        feat_label = ''
        for serie in transformed:
            if isinstance(Transformer, OneHotEncoder):
                feat_label = label + '_' + Transformer.active_features_[n]
            else:
                try:
                    feat_label = label + '_' + Transformer.classes_[n]
                except:
                    feat_label = label
            df[feat_label] = serie
            feature_list.append(feat_label)
        return df[feature_list]
    
    trn = train.copy()
    val = valid.copy()
    tst = test.copy()
    
    trn = _trans(trn, sCol, Transformer,'fit_transform')
    val = _trans(val, sCol, Transformer,'transform')
    tst = _trans(tst, sCol, Transformer,'transform')
    # If you have unseen data issue when transforming test, preprocess your data, don't leak. (Label don't exist in train
    
    #We only send back what we need
    return (trn, val, tst)


In [32]:
from sklearn.model_selection import train_test_split

X_train=df_train
y_train=df_train['interest_level']

x_trn, x_val, y_trn, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [33]:
x_trn

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
2683,1.0,0,ec12b4db154a9c75983f5e32edd93843,2016-06-28 05:51:34,Studio Apartment in Midtown East. Good lightin...,East 46th Street,"[Doorman, Elevator, Pre-War, Hardwood Floors, ...",low,40.7538,7226488,-73.9740,1331e8dd53e0ac1de44f92a492b5d240,[https://photos.renthop.com/2/7226488_2918a897...,2500,140 East 46th Street
38369,1.0,0,6f72525610dc82c4a486fc423637652f,2016-06-15 19:58:46,Welcome to the most exclusive home you will fi...,Financial District,"[Common Outdoor Space, Laundry In Unit, Cats A...",low,40.7048,7165750,-74.0102,0ab025a65730ab34eb292bd1dcbf67fa,[https://photos.renthop.com/2/7165750_ace07361...,3800,75 Wall
118394,1.0,3,1be9c496f7bdff4a552e3d4e91f915f8,2016-04-21 05:39:19,Bright and spacious Flex 3 apartment that feat...,Wall Street,"[Doorman, Elevator, Fitness Center, Laundry in...",low,40.7062,6907315,-74.0099,56d5b16222f138a23725efa0baee9889,[https://photos.renthop.com/2/6907315_296083e3...,3780,45 Wall Street
117210,1.0,1,d56c66e65b6f75b412a8353d3d1f0f4c,2016-04-04 07:22:49,Enjoy A Comfortable Lifestyle In This One Bedr...,"118th St, Kew Gardens, NY 11415","[Elevator, Laundry In Building]",low,40.7059,6822054,-73.8342,605757e0108c66606280905d76d50e80,[https://photos.renthop.com/2/6822054_6adbde20...,1800,"83-43 118th St, Kew Gardens, NY 11415"
70734,1.0,1,0,2016-05-10 04:57:58,,Saunders Street,"[Doorman, Elevator, Cats Allowed, Dogs Allowed...",low,40.7281,6990343,-73.8603,62826f3ae01f2ddc93b9cd28c659ab2b,[],1850,64-20 Saunders Street
13349,1.0,0,e03e0f9a61cfa10ca5e39c0310a238ee,2016-06-24 04:16:53,REMARKABLE studio on Upper East Side...short d...,E 84 Street,"[Hardwood Floors, Dogs Allowed, Cats Allowed]",low,40.7761,7208015,-73.9518,ed17ada77c1b94b4fb00d126a0c2ea06,[https://photos.renthop.com/2/7208015_6a2e70cd...,1875,324 E 84 Street
107516,1.0,1,5b2652311f5604103612368a57494ab3,2016-04-24 02:59:48,Stunning 1 Bd room available in the heart of C...,W 25th St.,"[Roof Deck, Doorman, Fitness Center, Laundry i...",low,40.7443,6918528,-73.9912,2645da97a81bfe1f4dbb98f3f4e167e0,[https://photos.renthop.com/2/6918528_98485f17...,5250,55 W 25th St.
114447,1.5,3,0,2016-04-06 07:44:28,"A wonderful blend of clean, modern townhouse l...",Kosciuszko,"[Fireplace, Garden/Patio, Multi-Level, Pre-War...",low,40.6903,6833758,-73.9530,7210d708363bfc63daeabf576a403046,[https://photos.renthop.com/2/6833758_6fcc0758...,3500,58 Kosciuszko
34325,1.0,2,80a120d6bc3aba97f40fee8c2204524b,2016-06-02 05:53:14,Spacious flex 2 units are newly renovated and ...,Water Street,"[Roof Deck, Doorman, Elevator, Laundry in Unit...",low,40.7073,7098839,-74.0046,69f035007d270281a624ab2cc1d2aba0,[https://photos.renthop.com/2/7098839_b52bd460...,3950,200 Water Street
75501,2.5,2,b70f31f5e92000a46f1db76f8947c6ec,2016-05-18 06:24:02,Extremely spacious 2 bedroom 2.5 bath converti...,W 55th St.,"[Fireplace, Dining Room, Doorman, Elevator, Fi...",low,40.7633,7031499,-73.9785,ad3d8ddc52c7e0859b5c6c7f7949c3bd,[https://photos.renthop.com/2/7031499_83fb5d8a...,6000,101 W 55th St.


In [ ]:
 sel = [
    (["bathrooms"],RobustScaler()),
    (["bedrooms"],RobustScaler()),
     ()
 ]

In [35]:
z =feat_selection(sel,x_trn, x_val,df_test)

[(['bathrooms'], RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), (['bedrooms'], RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True))]


In [37]:
z[2]

,bathrooms,bedrooms
0,0.0,0.0
1,0.0,1.0
100,0.0,0.0
1000,0.0,1.0
100000,1.0,1.0
100001,2.0,2.0
100002,0.0,1.0
100003,0.0,-1.0
100005,0.0,1.0
100006,0.0,-1.0
